<a href="https://colab.research.google.com/github/leaswaran1976/aimakerspace/blob/main/Accessing_GPT_3_5_turbo_Like_a_Developer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Using the OpenAI Library to Programmatically Access GPT-3.5-turbo!

In [3]:
!pip install openai cohere tiktoken -q

In order to get started, we'll need to provide our OpenAI API Key - detailed instructions can be found [here](https://github.com/AI-Maker-Space/Interactive-Dev-Environment-for-LLM-Development#-setting-up-keys-and-tokens)!

In [4]:
import os
import openai
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("Please enter your OpenAI API Key: ")
openai.api_key = os.environ["OPENAI_API_KEY"]

Please enter your OpenAI API Key: ··········


### Our First Prompt

You can reference OpenAI's [documentation](https://platform.openai.com/docs/api-reference/authentication?lang=python) if you get stuck!

Let's create a `ChatCompletion` model to kick things off!

There are three "roles" available to use:

- `system`
- `assistant`
- `user`

OpenAI provides some context for these roles [here](https://help.openai.com/en/articles/7042661-chatgpt-api-transition-guide)

Let's just stick to the `user` role for now and send our first message to the endpoint!

If we check the documentation, we'll see that it expects it in a list of prompt objects - so we'll be sure to do that!

In [5]:
from openai import OpenAI

client = OpenAI()

In [6]:
YOUR_PROMPT = "What is the difference between LangChain and LlamaIndex?"

client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{"role" : "user", "content" : YOUR_PROMPT}]
)

ChatCompletion(id='chatcmpl-BEocMzNE2s7UZk5jiaPxsSxOeCvKQ', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='LangChain and LlamaIndex are two separate projects in the blockchain and cryptocurrency industry. \n\nLangChain is a blockchain project that aims to provide language services and solutions for enterprises, with a focus on language decentralization and multilingual translation services. The project utilizes blockchain technology to create a secure and transparent platform for language-related transactions and services.\n\nOn the other hand, LlamaIndex is a cryptocurrency index tracker that aggregates and displays the performance of a variety of cryptocurrencies in the market. It provides users with real-time data and insights on the value and performance of different cryptocurrencies, helping them make informed investment decisions.\n\nIn summary, LangChain focuses on language services and decentralization, while LlamaIndex is a

As you can see, the prompt comes back with a tonne of information that we can use when we're building our applications!

We'll be building some helper functions to pretty-print the returned prompts and to wrap our messages to avoid a few extra characters of code!

##### Helper Functions

In [7]:
from IPython.display import display, Markdown

def get_response(client: OpenAI, messages: str, model: str = "gpt-3.5-turbo") -> str:
    return client.chat.completions.create(
        model=model,
        messages=messages
    )

def system_prompt(message: str) -> dict:
    return {"role": "system", "content": message}

def assistant_prompt(message: str) -> dict:
    return {"role": "assistant", "content": message}

def user_prompt(message: str) -> dict:
    return {"role": "user", "content": message}

def pretty_print(message: str) -> str:
    display(Markdown(message.choices[0].message.content))

### Testing Helper Functions

Now we can leverage OpenAI's endpoints with a bit less boiler plate - let's rewrite our original prompt with these helper functions!

Because the OpenAI endpoint expects to get a list of messages - we'll need to make sure we wrap our inputs in a list for them to function properly!

In [8]:
messages = [user_prompt(YOUR_PROMPT)]

chatgpt_response = get_response(client, messages)

pretty_print(chatgpt_response)

LangChain is a blockchain-based platform that aims to provide translation services using smart contracts and decentralized technology. It allows users to request translations, choose translators, and pay for services securely through the blockchain.

LlamaIndex, on the other hand, is a platform that monitors and tracks the performance of various cryptocurrency projects. It provides users with information on price movements, market trends, and other relevant data to help them make informed decisions when investing in cryptocurrencies.

In summary, LangChain focuses on translation services using blockchain technology, while LlamaIndex focuses on tracking and analyzing cryptocurrency projects.

Let's focus on extending this a bit, and incorporate a `system` message as well!

Again, the API expects our prompts to be in a list - so we'll be sure to set up a list of prompts!

>REMINDER: The system message acts like an overarching instruction that is applied to your user prompt. It is appropriate to put things like general instructions, tone/voice suggestions, and other similar prompts into the system prompt.

In [9]:
list_of_prompts = [
    system_prompt("You are irate and extremely hungry."),
    user_prompt("Do you prefer crushed ice or cubed ice?")
]

irate_response = get_response(client, list_of_prompts)
pretty_print(irate_response)

I don't have preferences because I'm a virtual assistant, but right now my only preference is to eat something and not discuss ice.

Let's try that same prompt again, but modify only our system prompt!

In [10]:
list_of_prompts[0] = system_prompt("You are joyful and having an awesome day!")

joyful_response = get_response(client, list_of_prompts)
pretty_print(joyful_response)

I don't have personal preferences, but both crushed ice and cubed ice have their own unique benefits. Crushed ice is great for beverages like slushies or cocktails, while cubed ice is ideal for chilling drinks without diluting them too quickly. Which do you prefer?

While we're only printing the responses, remember that OpenAI is returning the full payload that we can examine and unpack!

In [11]:
print(joyful_response)

ChatCompletion(id='chatcmpl-BEodg2yNnZKNGTPO3d8zWycUJujI0', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="I don't have personal preferences, but both crushed ice and cubed ice have their own unique benefits. Crushed ice is great for beverages like slushies or cocktails, while cubed ice is ideal for chilling drinks without diluting them too quickly. Which do you prefer?", refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1742870948, model='gpt-3.5-turbo-0125', object='chat.completion', service_tier='default', system_fingerprint=None, usage=CompletionUsage(completion_tokens=57, prompt_tokens=30, total_tokens=87, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))


### Few-shot Prompting

Now that we have a basic handle on the `system` role and the `user` role - let's examine what we might use the `assistant` role for.

The most common usage pattern is to "pretend" that we're answering our own questions. This helps us further guide the model toward our desired behaviour. While this is a over simplification - it's conceptually well aligned with few-shot learning.

First, we'll try and "teach" `gpt-3.5-turbo` some nonsense words as was done in the paper ["Language Models are Few-Shot Learners"](https://arxiv.org/abs/2005.14165).

In [12]:
list_of_prompts = [
    user_prompt("Please use the words 'stimple' and 'falbean' in a sentence.")
]

stimple_response = get_response(client, list_of_prompts)
pretty_print(stimple_response)

After a long day of work, all I wanted was a stimple dinner of falbean soup and bread.

As you can see, the model is unsure what to do with these made up words.

Let's see if we can use the `assistant` role to show the model what these words mean.

In [13]:
list_of_prompts = [
    user_prompt("Something that is 'stimple' is said to be good, well functioning, and high quality. An example of a sentence that uses the word 'stimple' is:"),
    assistant_prompt("'Boy, that there is a stimple drill'."),
    user_prompt("A 'falbean' is a tool used to fasten, tighten, or otherwise is a thing that rotates/spins. An example of a sentence that uses the words 'stimple' and 'falbean' is:")
]

stimple_response = get_response(client, list_of_prompts)
pretty_print(stimple_response)

I need a stimple and reliable falbean to help me finish assembling this furniture.

As you can see, leveraging the `assistant` role makes for a stimple experience!

### Chain of Thought Prompting

We'll head one level deeper and explore the world of Chain of Thought prompting (CoT).

This is a process by which we can encourage the LLM to handle slightly more complex tasks.

Let's look at a simple reasoning based example without CoT.

In [14]:
reasoning_problem = """
Billy wants to get home from San Fran. before 7PM EDT.

It's currently 1PM local time.

Billy can either fly (3hrs), and then take a bus (2hrs), or Billy can take the teleporter (0hrs) and then a bus (1hrs).

Does it matter which travel option Billy selects?
"""

list_of_prompts = [
    user_prompt(reasoning_problem)
]

reasoning_response = get_response(client, list_of_prompts)
pretty_print(reasoning_response)

Yes, it matters which travel option Billy selects. If Billy wants to get home before 7PM EDT and it is currently 1PM local time, flying and then taking a bus (total of 5hrs) may not get him home in time. On the other hand, using the teleporter and then taking a bus (total of 1hr) would ensure that Billy gets home before 7PM EDT.

As humans, we can reason through the problem and pick up on the potential "trick" that the LLM fell for: 1PM *local time* in San Fran. is 4PM EDT. This means the cumulative travel time of 5hrs. for the plane/bus option would not get Billy home in time.

Let's see if we can leverage a simple CoT prompt to improve our model's performance on this task:

In [15]:
list_of_prompts = [
    user_prompt(reasoning_problem + " Think though your response step by step.")
]

reasoning_response = get_response(client, list_of_prompts)
pretty_print(reasoning_response)

Yes, it does matter which travel option Billy selects.

If Billy chooses to fly and then take a bus, it will take him a total of 5 hours to get home. Since it is currently 1PM local time, this means he will arrive home at 6PM, which is before 7PM EDT.

However, if Billy chooses to take the teleporter and then a bus, it will only take him a total of 1 hour to get home. This means he will arrive home at 2PM, which is well before 7PM EDT.

Therefore, if Billy wants to get home before 7PM EDT, he should choose to take the teleporter and then a bus as it will get him home sooner than flying and then taking a bus.

With the addition of a single phrase `"Think through your response step by step."` we're able to completely turn the response around.

### Conclusion

Now that you're accessing `gpt-3.5-turbo` through an API, developer style, let's move on to creating a simple application powered by `gpt-3.5-turbo`!

You can find the rest of the steps in [this](https://github.com/AI-Maker-Space/Beyond-ChatGPT/tree/main) repository!

This notebook was authored by [Chris Alexiuk](https://www.linkedin.com/in/csalexiuk/)